# Okaloosa County FL SO - ALPR Audit Parser

## Instructions
1. Run ALL cells in order
2. Upload your PDF when prompted
3. Download the corrected CSV


## Step 1: Install Packages

## Step 2: Upload PDF

In [2]:
import os

# '../' goes up to the 'okaloosaalpr' root
# 'data/' goes into the data folder
filename = "../data/RAW-11_1_2025-12_8_2025-Okaloosa_County_FL_SO-Audit.pdf"

if os.path.exists(filename):
    print(f"✓ Found and linked: {filename}")
else:
    print(f"X Error: File not found at {os.path.abspath(filename)}")

✓ Found and linked: ../data/RAW-11_1_2025-12_8_2025-Okaloosa_County_FL_SO-Audit.pdf


## Step 3: Extract Text

In [3]:
import PyPDF2

with open(filename, 'rb') as f:
    reader = PyPDF2.PdfReader(f)
    num_pages = len(reader.pages)
    print(f"Extracting {num_pages} pages...")

    all_text = ""
    for i, page in enumerate(reader.pages):
        all_text += page.extract_text()
        if (i + 1) % 100 == 0:
            print(f"  {i + 1}/{num_pages}...")

all_lines = [line.strip() for line in all_text.split('\n') if line.strip()]
print(f"✓ Extracted {len(all_lines):,} lines")

Extracting 809 pages...
  100/809...
  200/809...
  300/809...
  400/809...
  500/809...
  600/809...
  700/809...
  800/809...
✓ Extracted 13,739 lines


## Step 4: Preprocess - Split Concatenated UUIDs

In [4]:
import re

uuid_pattern = r'[a-f0-9]{8}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{12}'

print("Splitting concatenated UUIDs...")
final_lines = []
splits = 0

for line in all_lines:
    uuid_matches = list(re.finditer(uuid_pattern, line))

    if len(uuid_matches) == 0:
        final_lines.append(line)
    elif len(uuid_matches) == 1:
        uuid_start = uuid_matches[0].start()
        if uuid_start == 0 or line[uuid_start - 1].isspace():
            final_lines.append(line)
        else:
            splits += 1
            final_lines.append(line[:uuid_start].rstrip())
            final_lines.append(line[uuid_start:])
    else:
        # Multiple UUIDs - split at each boundary
        splits += 1
        for j, match in enumerate(uuid_matches):
            uuid_start = match.start()
            if j == 0:
                if uuid_start > 0 and not line[uuid_start - 1].isspace():
                    final_lines.append(line[:uuid_start].rstrip())
                if j + 1 < len(uuid_matches):
                    final_lines.append(line[uuid_start:uuid_matches[j + 1].start()].rstrip())
                else:
                    final_lines.append(line[uuid_start:])
            else:
                if j + 1 < len(uuid_matches):
                    final_lines.append(line[uuid_start:uuid_matches[j + 1].start()].rstrip())
                else:
                    final_lines.append(line[uuid_start:])

all_lines = final_lines
print(f"✓ Preprocessed: {len(all_lines):,} lines ({splits} splits)")

Splitting concatenated UUIDs...
✓ Preprocessed: 14,547 lines (808 splits)


## Step 5: Parse Records

In [5]:
import pandas as pd

def is_uuid_line(line):
    parts = line.split()
    if parts:
        return bool(re.match(r'^' + uuid_pattern + r'$', parts[0]))
    return False

records = []
i = 0

while i < len(all_lines):
    try:
        line = all_lines[i]

        if not is_uuid_line(line):
            i += 1
            continue

        parts = line.split(' ')
        devices = parts[6] if len(parts) > 6 else ''
        if '/' in devices:
            match = re.match(r'(\d+)', devices)
            devices = match.group(1) if match else devices

        record = {
            'ID': parts[0],
            'Org_Name': ' '.join(parts[1:5]),
            'Total_Networks_Searched': parts[5] if len(parts) > 5 else '',
            'Total_Devices_Searched': devices
        }

        if i + 1 >= len(all_lines) or is_uuid_line(all_lines[i + 1]):
            record.update({
                'Time_Frame_Start': '', 'Time_Frame_End': '', 'Reason': '',
                'Search_Time': '', 'Search_Type': '', 'Moderation': ''
            })
            records.append(record)
            i += 1
            continue

        line2 = all_lines[i + 1]
        utc_splits = line2.split('UTC')

        if len(utc_splits) >= 1:
            time_frame_start = utc_splits[0].strip()
            if not re.search(r'\d{2}/\d{2}/\d{4}', time_frame_start):
                date_in_line1 = re.search(r'(\d{2}/\d{2}/\d{4},)\s*$', line)
                if date_in_line1:
                    time_frame_start = date_in_line1.group(1) + ' ' + time_frame_start
            record['Time_Frame_Start'] = time_frame_start + ' UTC'
        else:
            record['Time_Frame_Start'] = ''

        if len(utc_splits) >= 2:
            end_datetime = utc_splits[1].strip()
            datetime_match = re.match(r'(\d{2}/\d{2}/\d{4},\s+\d{2}:\d{2}:\d{2}\s+[AP]M)', end_datetime)
            record['Time_Frame_End'] = datetime_match.group(1) + ' UTC' if datetime_match else ''
        else:
            record['Time_Frame_End'] = ''

        if len(utc_splits) >= 3:
            reason_and_search = utc_splits[2].strip()
            search_time_match = re.search(r'(\d{2}/\d{2}/\d{4},\s+\d{2}:\d{2}:\d{2}\s+[AP]M)', reason_and_search)
            if search_time_match:
                record['Reason'] = reason_and_search[:search_time_match.start()].strip()
                record['Search_Time'] = search_time_match.group(1) + ' UTC'
            else:
                record['Reason'] = reason_and_search
                record['Search_Time'] = ''
        else:
            record['Reason'] = ''
            record['Search_Time'] = ''

        if len(utc_splits) >= 4:
            search_type_part = utc_splits[3].strip()
            parts = search_type_part.split()
            if parts:
                record['Search_Type'] = parts[0]
                record['Moderation'] = ' '.join(parts[1:]) if len(parts) > 1 else ''
            else:
                record['Search_Type'] = ''
                record['Moderation'] = ''
        else:
            record['Search_Type'] = ''
            record['Moderation'] = ''

        records.append(record)
        i += 2

        if len(records) % 500 == 0:
            print(f"  Processed {len(records):,}...")
    except Exception as e:
        i += 1
        continue

print(f"✓ Parsed {len(records):,} records")

  Processed 500...
  Processed 1,000...
  Processed 1,500...
  Processed 2,000...
  Processed 2,500...
  Processed 3,000...
  Processed 3,500...
  Processed 4,000...
  Processed 4,500...
  Processed 5,000...
  Processed 5,500...
  Processed 6,000...
  Processed 6,500...
  Processed 7,000...
✓ Parsed 7,273 records


In [10]:
# --- Step 5.5: Consistent Month-Suffix Correction ---
print("Starting consistent suffix correction...")

corrected_count = 0

for record in records:
    devices = str(record.get('Total_Devices_Searched', ""))
    start_date = str(record.get('Time_Frame_Start', ""))

    # We need at least 2 digits for a month and some device digits to work with
    if len(start_date) >= 2 and len(devices) > 2:
        # 1. Get the month digits from the start of the date (e.g., "11")
        # We split by '/' to handle both "11/" and "1/" cases safely
        month_prefix = start_date.split('/')[0]

        # Ensure month_prefix is 2 digits if it's 10, 11, or 12
        # (The parser consistently grabs 2 digits in the broken rows)
        if len(month_prefix) == 2:
            # 2. If the device count ends with those exact digits, strip them
            if devices.endswith(month_prefix):
                record['Total_Devices_Searched'] = devices[:-2]
                corrected_count += 1
        elif len(month_prefix) == 1:
            # Handle single digit months if they are also duplicating (e.g., "9")
            if devices.endswith(month_prefix):
                record['Total_Devices_Searched'] = devices[:-1]
                corrected_count += 1

print(f"✓ Fixed {corrected_count} rows by removing duplicated month suffixes.")

Starting consistent suffix correction...
✓ Fixed 51 rows by removing duplicated month suffixes.


## Step 6: Export

In [11]:
import os

df = pd.DataFrame(records)

print("\nValidation:")
print(f"  Total records: {len(df):,}")
print(f"  Valid UUIDs: {df['ID'].str.match(uuid_pattern, na=False).sum():,}")
print(f"  Complete records: {((df['Time_Frame_Start'] != '') & (df['Time_Frame_End'] != '') & (df['Search_Time'] != '')).sum():,}")

print("\nFirst 3 records:")
print(df.head(3))




import os

# 1. Get just the filename without the "../data/" prefix
file_only = os.path.basename(filename) 

# 2. Get the name without the ".pdf" extension
name_no_ext = os.path.splitext(file_only)[0]

# 3. Construct the new path pointing to the 'exports' folder
output_filename = f"../exports/{name_no_ext}_CORRECTED.csv"

# 4. Save
df.to_csv(output_filename, index=False)

print(f"✓ File saved to: {output_filename}")




base_name = os.path.splitext(filename)[0]
output_filename = f"{base_name}_CORRECTED.csv"
df.to_csv(output_filename, index=False)

print(f"\n✓ Exported: {output_filename}")



Validation:
  Total records: 7,273
  Valid UUIDs: 7,273
  Complete records: 7,273

First 3 records:
                                     ID               Org_Name  \
0  23d55b89-42eb-4f5c-9281-f5e8bf4864be  Okaloosa County FL SO   
1  5720871a-4c6d-4be1-a9c8-6f7524f871f9  Okaloosa County FL SO   
2  50bac7fa-a41a-45de-86c8-622fcfa0a0f5  Okaloosa County FL SO   

  Total_Networks_Searched Total_Devices_Searched             Time_Frame_Start  \
0                     840                  17652  11/13/2025, 10:00:17 PM UTC   
1                     840                  17652  11/13/2025, 10:30:13 PM UTC   
2                     840                  17652  11/13/2025, 11:00:22 PM UTC   

                Time_Frame_End Reason                  Search_Time  \
0  11/14/2025, 10:00:17 AM UTC   bolo  11/14/2025, 09:53:36 AM UTC   
1  11/14/2025, 10:30:13 AM UTC   bolo  11/14/2025, 10:27:32 AM UTC   
2  11/14/2025, 11:00:22 AM UTC   bolo  11/14/2025, 10:48:23 AM UTC   

  Search_Type Moderation  
0